In [1]:
from tabulate import tabulate

In [2]:
def sigmoid(z):
    e = 2.71828
    F = 1/(1 + e**(-z))
    return F

In [3]:
class Neuron(object):

    def __init__(self, bias, weights):
        self.bias = bias
        self.weights = weights
        self.outputs = None
        self.errors = None
        self.deltaWeight = []
        self.deltaBias = None
        self.hiddenError = None
        self.previousLayer = None
        self.hidden = True
        
    # Deze functie geeft de eigenschappen van de neuron weer.   
    def __str__(self): 
        return 'Neuron(Bias=' + str(self.bias) + ', Weights=' + str(self.weights) + ', inputs=' + str(self.inputs) + ', outputs=' + str(self.outputs)+ ', Errors=' + str(self.errors)
    
    # Haalt de errors op van de neuron
    def get_errors(self):
        return self.errors
    
    # Haalt de hiddenErrors op van de neuron
    def get_hiddenError(self):
        return self.hiddenError
    
    # Activatie functie van de neuron
    def activate(self, inputs):
        total = 0
        self.inputs = inputs
        
        # Berekent de som en telt de bias er bij op
        for i in range(len(self.inputs)):
            total += self.inputs[i] * self.weights[i]
        
        total = total + self.bias
        
        # Voert de sigmoid functie uit over het totaal
        self.outputs = sigmoid(total)
        return sigmoid(total)
    
    # Berekent de error van een neuron in de output laag
    def error(self, target):
        error = self.outputs * (1-self.outputs) * -(target - self.outputs)
        self.errors = error
        return error
    
    def gradient(self, connectedNeuron):
        gradient = self.outputs * connectedNeuron.get_errors
        self.gradient[i] = gradient
        return gradient
    
    
    # berekent de aanpassing aan een weight.
    def deltaWeights(self, learningrate):
        self.deltaWeight = []
        
        # Loopt over de vorige laag heen en kent een specifieke neuron toe.
        for i in range(len(self.previousLayer.neurons)):
            connectedNeuron = self.previousLayer.neurons[i]
            
            # Loopt over de weights
            for j in range(len(self.weights)):
                
                # Checkt of de neuron waar deze aan vast zit een Error heeft (een output neuron is)
                if connectedNeuron.hidden == False:
                    deltaweight = learningrate * self.outputs * connectedNeuron.get_errors()
                    self.deltaWeight.append(deltaweight)
                
                # Als connectedNeuron.hidden True is hebben we te maken met een hidden neuron en gebruikt hij de hidden error
                else:
                    deltaweight = learningrate * self.outputs * connectedNeuron.get_hiddenError()
                    self.deltaWeight.append(deltaweight)
                

    # berekent het verschil in de bias        
    def delta_Bias(self, learningrate):
        # Checkt of het een output of hidden neuron is en aan de hand daarvan kiest hij welke error hij gebruikt.
        if self.hidden == False:
            self.deltaBias = learningrate * self.get_errors()
        else:
            self.deltaBias = learningrate * get_hiddenError()
            
    # Voert de veranderingen door aan de weights en biases.
    def update(self):
        print(self.weights, self.bias) # nog voor testen
        
        # loopt over alle weights en over de bias en voert de verandering door.
        for i in range(len(self.weights)):
            self.weights[i]= self.weights[i] - self.deltaWeight[i]
        self.bias = self.bias - self.deltaBias
    
    # Berekent de error van een hiddenNeuron doormiddel van de afgeleide sigmoid. 
    def errorHiddenNeuron(self, connectedNeurons):
        total = 0
        for i in range(len(self.weights)):
            connectedNeuron = connectedNeurons.neurons[i]
            total += connectedNeuron.get_errors() * self.weights[i]
            self.hiddenError = self.outputs * (1-self.outputs) * total
        

In [4]:
class Neuronlayer(object):
    
    
    def __init__(self, neurons):
        self.neurons = neurons     
    
    # Activatie functie van een Neuronlayer
    def activate(self, inputs):
        total = []
        
        #ittereert over de neurons
        for i in range(len(self.neurons)):
            # Saved welke neuron er aan de beurt is
            neuron = self.neurons[i]   
            
            #Neemt de input en voert de activate functie uit met de waardes die die neuron heeft
            total.append(neuron.activate(inputs))
              
        return total
    
    

    
    

In [5]:
class NeuronNetwork(object):
    
    
    def __init__(self, neuronLayers):
        self.neuronLayers = neuronLayers
        self.neuronLayer = None
    
    # De feedfoward functie van het netwerk
    def feed_foward(self, inputs):
        # Loopt over de layers heen en houdt bij, bij welke layer hij is.
        for i in range(len(self.neuronLayers)):
            neuronLayer = self.neuronLayers[i]   
            
            # output is de uitkomst van de neuronlayer activatie functie de gegeven inputs
            output = neuronLayer.activate(inputs)
            
            # Maakt van de output de nieuwe input voor de volgende iteratie.
            inputs = output
            
        return output
    
    # Implementatie van backwardpropegation
    def propegateBackward(self, target, learningrate):
        # Loopt door de layers van achter naar voren en kent de layer toe.
        for i in range(len(self.neuronLayers[::-1])):
            self.neuronLayer = self.neuronLayers[i]
            
            # Loopt over de neurons in de neuronlayer.
            for j in range(len(self.neuronLayer.neurons)):
                neuron = self.neuronLayer.neurons[j]
                
                # Als i nog 0 is zitten we in de output laag en hebben te maken met een output layer
                # en gebruikt hij de functie neuron.error(target)
                if i == 0:
                    neuron.hidden = False
                    neuron.error(target)
                
                # als i hoger is dan 0 zitten we in een hiddenlayer en gebruikt hij de functie neuron.errorHiddenNeuron 
                elif i > 0:
                    neuron.hidden = True
                    neuron.errorHiddenNeuron(self.neuronLayers[i-1])
               
                neuron.previousLayer = self.neuronLayers[i-1]
                
                # berekent de het verschil in de weights en bias's
                neuron.deltaWeights(learningrate)
                neuron.delta_Bias(learningrate)
        
    
    # Deze functie past alle veranderingen toe op de neuronen in het netwerk
    def updateNetwork(self):
        for i in range(len(self.neuronLayers[::-1])):

            self.neuronLayer = self.neuronLayers[i]
            
            for j in range(len(self.neuronLayer.neurons)):
                
                neuron = self.neuronLayer.neurons[j]
               
                neuron.update()
        

In [6]:
Hiddenlayer1 = Neuronlayer([Neuron(1.5 ,[-0.5, 0.5])])

In [7]:
andGate = NeuronNetwork([Hiddenlayer1],)
inputs = [[0,0],[1,0],[0,1],[1,1]]
results = []
target = [0,0,0,1]
learningrate = 1
pair = []

for j in range(1):
    for i in range(len(inputs)):
        pair.append(inputs[i])
    
        pair.append(andGate.feed_foward(inputs[i]))
        andGate.propegateBackward(target[i], learningrate)
        andGate.updateNetwork() # print nog de weights en bias'es van het testen
        
        # gemiddelde error berekenen en stopconditie toevoegen


        
    

    


   



[-0.5, 0.5] 1.5
[-0.5996936954512968, 0.40030630454870314] 1.3780616118724482
[-0.7009806708578037, 0.29901932914219626] 1.2302682132992213
[-0.7998631259424432, 0.20013687405755665] 1.1099588974992503
